In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

outputs_dir = Path('./outputs/0126/test1')
MAX_LATENCY_MS = 120000

def parse_receiver_log(f, marker="P4 is cool"):
    ts, pending = [], None
    if not Path(f).exists(): return ts
    for line in open(f, 'r'):
        if 'got a packet' in line: pending = None; continue
        m = re.search(r'received at time : ([\d.]+)', line)
        if m: pending = float(m.group(1)); continue
        if marker in line and pending: ts.append(pending); pending = None
    return ts

def parse_sender_log(f):
    out = []
    if not Path(f).exists(): return out
    for line in open(f, 'r'):
        m = re.search(r'sent (\d+) packets until now : ([\d.]+)', line)
        if m: out.append((int(m.group(1)), float(m.group(2))))
    return out

def calc_latency_first_layer(send_times, recv_times):
    """第一层精确索引配对: 第i个recv <-> 第i个send"""
    lat = []
    for i, rt in enumerate(recv_times):
        if i >= len(send_times): break
        _, st = send_times[i]
        l = (rt - st) * 1000
        if 0 <= l < MAX_LATENCY_MS: lat.append(l)
    return lat

flow_latencies = {}
for fid in range(3):
    send = parse_sender_log(outputs_dir / f"sender_h{fid+1}.txt")
    recv = parse_receiver_log(outputs_dir / f"receiver_h_r{fid+1}.txt")
    flow_latencies[fid] = calc_latency_first_layer(send, recv)
    print(f"Flow {fid}: sent={len(send)}, recv={len(recv)}, latencies={len(flow_latencies[fid])}")

In [ ]:
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
labels = ['Flow 0 (High Weight)', 'Flow 1 (Medium Weight)', 'Flow 2 (Low Weight)']
plt.figure(figsize=(10, 6))
for idx in range(3):
    lat = flow_latencies[idx]
    if lat:
        s = np.sort(lat)
        plt.plot(s, np.arange(1, len(s)+1)/len(s), label=labels[idx], color=colors[idx], linewidth=2)
plt.xlabel('Latency (ms)', fontsize=12); plt.ylabel('CDF', fontsize=12)
plt.title('WRR Latency CDF', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3); plt.legend(fontsize=11); plt.tight_layout(); plt.show()